In [1]:
## Calculate selected properties in Qsg cases
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os, sys
sys.path.append(os.path.abspath('../TEF'))
import TEF_Variables as tef

In [ ]:
datapath = '/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_nosill'
# case_id = 1
# file0 = xr.open_dataset(datapath+'/icefrntM_' + str(format(case_id,'03d')) + '.nc')
# file = file0.isel(T=~file0.get_index("T").duplicated()) 

# pm = file.icefrntM.data
# # t0 = 0
# # tn = len(file.T)
# # state = file.isel(Y=range(35,45), T=range(t0,tn))

# pm[2,:,40, 1]

In [2]:
# Obtain reflux coefficients and entrainment flux Qe
datapath = '/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_minhs'
def reflux(case_id, xi):

    State0 = xr.open_dataset(datapath+'/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(datapath+'/grid_' + str(format(case_id,'03d')) + '.nc')
    State = State0.isel(T=~State0.get_index("T").duplicated())
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))


    s = state.S.data.mean(2) # Salinity in T,Z,X
    u = (state.U.data[:,:,:,1:].mean(2) + state.U.data[:,:,:,:-1].mean(2)) / 2 # Along channel velocity
    ot = state.T.data # Time in seconds


    HFacC1 = grid.HFacC.data.mean(1)
    dyF1 = grid.dyF.data.mean(0)
    drF1 = grid.drF.data
    gridA = np.broadcast_to(drF1[:, np.newaxis], HFacC1.shape) * np.broadcast_to(dyF1[np.newaxis, :], HFacC1.shape) * HFacC1 # Grid Area
    da = np.broadcast_to(gridA[np.newaxis,:,:], u.shape)

    S = state.S.data
    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
    
    
    Qin = np.empty(len(xi))
    Qout = np.empty(len(xi))
    Sin = np.empty(len(xi))
    Sout = np.empty(len(xi))
    t0 = 89 # 274
#    Exclude the upper (surface) layer for TEF calculation
#     for t in range(U.shape[0]):
        
#         Uzx = u[t,:,:] # Steady-state along-channel velocity
#         for j in range(Uzx.shape[1]):
#             if any(Uzx[:,j]>0)==False:
#                 pass
#             else:
#                 mid = np.where(Uzx[:,j]>0)[0][0]
#                 if mid>0:
#                     S[t,:mid,:,j] = 0
#                     U[t,:mid,:,j] = 0
#                 else:
#                     pass
    
    Uzx = u[t0:,:,:].mean(0) # Steady-state along-channel velocity
    for j in range(Uzx.shape[1]):
        if any(Uzx[:,j]>0)==False:
            pass
        else:
            mid = np.where(Uzx[:,j]>0)[0][0]
            if mid>0:
                S[:,:mid,:,j] = 0
                U[:,:mid,:,j] = 0
            else:
                pass
    
    for i in range(len(xi)):
    
        tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1 = tef.process_section(U,S,DA,ot,xi[i],23,testing=False)

        qin1, qout1, sin1, sout1 = tef.bulk_calc(tef_q1, tef_vel1, tef_da1, tef_qs1, tef_qs21, sbins1, ot)
    
        Qin[i] = qin1[t0:].mean() # Time averaging from 1+t0 hour
        Qout[i] = qout1[t0:].mean()
        Sin[i] = sin1[t0:].mean() # Time averaging from 1+t0 hour
        Sout[i] = sout1[t0:].mean()
        
    a002 = (Sin[0]/Sout[0])*(Sout[1]-Sout[0])/(Sout[1]-Sin[0])
    a001 = (Qin[0]-Qin[1]) / Qin[0]    
    a11 = (Sout[1]/Sin[1])*(Sin[1]-Sin[0])/(Sout[1]-Sin[0])    
    a01 = (Sout[1]/Sout[0])*(Sout[0]-Sin[0])/(Sout[1]-Sin[0])
    
    a10 = (Sin[0]/Sin[1])*(Sout[1]-Sin[1])/(Sout[1]-Sin[0])
    
    #a00 = -(Qout[0]/Qin[0])*(Sout[1]-Sout[0])/(Sout[1]-Sin[0])
    #a11 = -(Qin[1]/Qout[1])*(Sin[1]-Sin[0])/(Sout[1]-Sin[0])
    #a01 = (Qin[1]/Qin[0])*(Sout[1]-Sin[1])/(Sout[1]-Sin[0])
    #a10 = (Qout[0]/Qout[1])*(Sout[0]-Sin[0])/(Sout[1]-Sin[0])
    
    q0 = Qin[0]
    q1 = -Qout[-1]
    Q0 = -Qout[0]
    Q1 = Qin[-1]
    f0 = Sin[0]*q0
    f1 = Sout[-1]*q1
    F0 = Sout[0]*Q0
    F1 = Sin[-1]*Q1
    
    A = np.array([[q1, q0, 0, 0], [f1, f0, 0, 0], [0, 0, q1, q0], [0, 0, f1, f0]])
    B = np.array([Q1, F1, Q0, F0])
    X = np.linalg.solve(A,B) # a11, a01, a10, a00
    
    Qe = Q0 - q1
    
#     if (a11 < 0.1) or (a11 > 1):
#         a22 = a001
#     else:
#         a22 = a002
    if (X[-1] < 0) or (X[-1] > 1):
        a22 = a001
        a11 = 0
    else:
        a22 = X[-1]
        a11 = X[0]
    
    return a11,a22,q1,q0,Q1,Q0
    #return a11,a22,q1,q0,Q1,Q0

In [3]:
def vol_temp(case_id, xi):
    
    State0 = xr.open_dataset(datapath+'/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(datapath+'/grid_' + str(format(case_id,'03d')) + '.nc')
    State = State0.isel(T=~State0.get_index("T").duplicated())    
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2 # Along-channel velocity
    
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
    
    rA = np.broadcast_to(grid.rA.data[np.newaxis, np.newaxis, :, :], U.shape)
    CV = rA * drF * HFacC

    
    t0 = 89 # 274
    da = DA[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    cv = CV[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    s = state.S.data[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    temp = state.Temp.data[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    u = U[t0:,:,:,xi[0]+1:xi[1]+1].mean(axis=(0,2))
    sma = np.ma.masked_where(s==0, s)
    topo = np.ma.getmask(sma) # Masked Topography
    uma = np.ma.MaskedArray(u, mask=topo)
    tma = np.ma.MaskedArray(temp, mask=topo)
    
    
    Tv1 = np.empty(uma.shape[1])
    Tv2 = np.empty(uma.shape[1])
    Vol1 = np.empty(uma.shape[1])
    Vol2 = np.empty(uma.shape[1])
    Tvol1 = np.empty(uma.shape[1])
    Tvol2 = np.empty(uma.shape[1])
    L = np.empty(uma.shape[1])
    
    for i in range(uma.shape[1]):

        # Exclude the surface layer 
        if any(uma[:,i]>0)==False:
            pass
        else:
            mid = np.where(uma[:,i]>0)[0][0]
            if mid>0:
                uma[:mid,i] = 0
            else:
                pass
            
        if any(uma[:,i]<0)==False:
            pass
            
        else:
            #meanS = np.abs(sma[:,i] - (sma[:,i].max() + sma[:,i].min()) / 2)
            #l = np.argmin(meanS)
            l = np.where(uma[:,i] < 0)[-1][0]    
            Tv1[i] = np.sum(tma[:l,i]*da[:l,i]*uma[:l,i]) / np.sum(da[:l,i]*uma[:l,i])
            Tv2[i] = np.sum(tma[l:,i]*da[l:,i]*uma[l:,i]) / np.sum(da[l:,i]*uma[l:,i])
            Vol1[i] = cv[:l,i].sum()
            Vol2[i] = cv[l:,i].sum()
            Tvol1[i] = np.sum(tma[:l,i]*cv[:l,i])
            Tvol2[i] = np.sum(tma[l:,i]*cv[l:,i])
            L[i] = l
            
        Tf = Tv1[:-1].mean()
        Ts = Tv2[:-1].mean()
        #Tf = Tvol1[:-1].sum()/Vol1[:-1].sum()
        #Ts = Tvol2[:-1].sum()/Vol2[:-1].sum()
        Ts_in = Tv2[-1]
        
    return Tf, Ts, Ts_in
    #return L

In [4]:
# Obtain submarine melting Qsm
def IFA(case_id):
    # Grid areas
    Area = np.empty([90, 10])
    Area[:20,:] = 400
    Area[20:50,:] = 800
    Area[50:,:] = 1200
    file0 = xr.open_dataset(datapath+'/icefrntA_' + str(format(case_id,'03d')) + '.nc')
    file = file0.isel(T=~file0.get_index("T").duplicated()) 
    t0 = 0
    #t0 = 89
    tn = len(file.T)
    state = file.isel(Y=range(35,45), T=range(t0,tn))
    MR = state.icefrntA.isel(X=1).data.mean(0) # Melt rate at the icefront
    Qsm = (MR*Area).sum()/(24*3600)
    return Qsm

In [ ]:
Qsg = np.array([25,50,100,250,500,1000])

case = np.array([1,2,3,4,5])

Qsm = np.empty(len(case))

for j in range(len(case)):

    Qsm[j] = IFA(case[j])
    
    
Qsm

In [ ]:
TEF_ds = xr.Dataset(
    data_vars={'Qsg' : Qsg,
    'alpha11' : alp11,
    'alpha22' : alp22,
    'Qin1' : Qout0,
    'Qin2' : Qin1,
    'Ts_in' : Ts_in,
    'Ts_e' : Tse,
    'Ts' : Ts,
    'Qsm' : Qsm})


outdir = "/home/1959/Parameters/FjordModeling/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

TEF_ds.to_netcdf(outdir + 'Qsg_minhs_qsm0.nc')

In [5]:
Qsg = np.array([25,50,100,250,500,1000])

case = np.array([1,2,3,4,5,6])
xrange = np.array([120,235]) # X index range for the sill segment

alp11 = np.empty(len(case))
alp22 = np.empty(len(case))
Qin0 = np.empty(len(case))
Qin1 = np.empty(len(case))
Qout0 = np.empty(len(case))
Qout1 = np.empty(len(case))
Ts = np.empty(len(case))
Tf = np.empty(len(case))
Ts_in = np.empty(len(case))
Qsm = np.empty(len(case))

for j in range(len(case)):

    alp11[j], alp22[j], Qin1[j], Qin0[j], Qout1[j], Qout0[j] = reflux(case[j],xrange)
    Tf[j], Ts[j], Ts_in[j] = vol_temp(case[j],xrange)
    Qsm[j] = IFA(case[j])


Tse = (Ts_in*Qin1*(1-alp11) + Tf*Qin0*alp22) / Qout0
Tr = Tse / Ts_in
Qe = Qout0 - Qin1


  time 0 out of 120
  time 0 out of 120


/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:269: RuntimeWarning: invalid value encountered in true_divide
  Sm = QSm/Qm
/home/1959/Scripts/Jupyter-Iceplume/TEF/TEF_Variables.py:273: RuntimeWarning: invalid value encountered in true_divide
  S2m = QS2m/Qm


  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120
  time 0 out of 120


In [6]:
Tse, Ts_in, Tr

(array([4.99399311, 6.98762798, 8.38650404, 9.00732925, 8.86180463,
        8.45535159]),
 array([9.99331503, 9.96899144, 9.90174166, 9.95629743, 9.9745577 ,
        9.98320485]),
 array([0.49973338, 0.7009363 , 0.84697262, 0.90468664, 0.88844086,
        0.84695764]))

In [7]:
# Save outputs

TEF_ds = xr.Dataset(
    data_vars={'Qsg' : Qsg,
    'alpha11' : alp11,
    'alpha22' : alp22,
    'Qin1' : Qout0,
    'Qin2' : Qin1,
    'Ts_in' : Ts_in,
    'Ts_e' : Tse,
    'Ts' : Ts,
    'Qsm' : Qsm})

outdir = "/home/1959/Parameters/FjordModeling/"
if not os.path.exists(outdir):
    os.makedirs(outdir)

TEF_ds.to_netcdf(outdir + 'Qsg_minhs_qsm0.nc')
